In [ ]:
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored  

In [ ]:
API_KEY = '5mdim-8qu4h-pq2g7-ivhl6-memyi'

Create an OpenAI compatible API client for making calls into one of the HPC's language model.

In [ ]:
client = OpenAI(
    api_key=API_KEY,
    base_url="https://api.hpc.inl.gov/llm/v1",
)

Obtain the list of language models that can be used.

In [ ]:
model_list = client.models.list()
print(f'model_list: {model_list}')
model_names = [x.id for x in model_list.data]
print(f'model names: {model_names}')

In [ ]:
GPT_MODEL = model_names[0]

Define a few utilities for making calls to the Chat Completions API and for maintaining and keeping track of the conversation state.

In [ ]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [ ]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))

In [ ]:
systemRoleContent= "You are a technical assistant, skilled in explaining complex programming concepts with everyday language."

descriptionPrompt = "Given the following list of variable names, generate a sentence description for each."

friendlyNamesPrompt = """Given the following list of variable names, separate each variable name into words with a space between words. 
     Search through each variable name to determine if any substring matches a well known english word.
     Interpret the use of camel case or Pascal case within a variable name as defining a word boundary.
     Interpret the _ character within a variable name as defining a word boundary.
     For example ApprRequestForm_ID would be separated into the following words: Appr Request Form ID"""

In [ ]:
exampleColumnNames = """
AccessRequest
ApprRequestForm_ID
Area_ID
Building
Escort_ID
LOGEndDate
LOGStartDate
Purpose
Request_Area_ID
Room
ApprRequestForm_ID
Comments
DtAction
LOGEndDate
LOGStartDate
NTID
Other_ID
Request_StateAction_ID
SNumber
StateAction_ID
"""

In [ ]:
# Generate element level descriptions
completion = client.chat.completions.create(model=GPT_MODEL,
  messages=[
    {"role": "system", "content": systemRoleContent},
    {"role": "user", "content": descriptionPrompt + exampleColumnNames}
  ]
)
descriptions = completion.choices[0].message.content

# Generate friendly names
completion = client.chat.completions.create(model=GPT_MODEL,
  messages=[
    {"role": "system", "content": systemRoleContent},
    {"role": "user", "content": friendlyNamesPrompt + exampleColumnNames}
  ]
)
friendlyNames = completion.choices[0].message.content


print(f"Descriptions:\n{descriptions.replace('his variable ', 'his element ')}")
print(f"\nFriendly Names:\n{friendlyNames}")